In [46]:
%%writefile gen_plot_data.py 

import sys
from bench_basic import sum2d
from numba import njit
import pickle

disp = njit(sum2d)
disp.compile("(intp,)")
md = disp.get_metadata(disp.signatures[0])
pipeline_timings = md["pipeline_times"]["nopython"]
data = {'_'.join(k.split('_', 1)[1:]): sum(v) for k, v in pipeline_timings.items()}

with open(sys.argv[1], "wb") as fout:
    pickle.dump(data, fout)

Overwriting gen_plot_data.py


In [47]:
!NUMBA_USE_RVSDG_FRONTEND=1 python gen_plot_data.py plot_rvsdg.dat

In [48]:
!NUMBA_USE_RVSDG_FRONTEND=0 python gen_plot_data.py plot_baseline.dat

In [49]:
import pickle
data_rvsdg = pickle.load(open("plot_rvsdg.dat", "rb"))
data_baseline = pickle.load(open("plot_baseline.dat", "rb"))

In [50]:
import pandas as pd

In [51]:
df_baseline = pd.DataFrame({
        "run": ["baseline"] * len(data_baseline),
        "pass": [*data_baseline.keys()], 
        "time": [*data_baseline.values()],
    })
df_rvsdg = pd.DataFrame({
        "run": ["rvsdg"] * len(data_rvsdg),
        "pass": [*data_rvsdg.keys()], 
        "time": [*data_rvsdg.values()],
    })




df = pd.concat([df_baseline, df_rvsdg])
df = df.groupby("run").apply(lambda x: x[["pass", "time"]].set_index("pass"))
df

time
run      pass                                           
baseline translate_bytecode                     0.002256
         fixup_args                             0.000003
         ir_processing                          0.000559
         with_lifting                           0.000087
         inline_closure_likes                   0.002133
         rewrite_semantic_constants             0.000035
         dead_branch_prune                      0.000057
         generic_rewrites                       0.000689
         Rewrite dynamic raises                 0.000012
         make_function_op_code_to_jit_function  0.000016
         inline_inlinables                      0.000044
         find_literally                         0.000024
         literal_unroll                         0.000017
         reconstruct_ssa                        0.001631
         LiteralPropagation                     0.000018
         nopython_type_inference                0.005675
         strip_phis                             0.000684
         inline_overloads                       0.000190
         nopython_rewrites                      0.001845
         nopython_supported_feature_validation  0.000058
         ir_legalization                        0.000683
         annotate_types                         0.000035
         native_lowering                        0.044012
         nopython_backend                       0.000022
         dump_parfor_diagnostics                0.000003
rvsdg    rvsdg_frontend                         0.027558
         fixup_args                             0.000006
         ir_processing                          0.000868
         with_lifting                           0.000114
         inline_closure_likes                   0.002875
         rewrite_semantic_constants             0.000047
         dead_branch_prune                      0.000075
         generic_rewrites                       0.000957
         Rewrite dynamic raises                 0.000018
         make_function_op_code_to_jit_function  0.000019
         inline_inlinables                      0.000065
         find_literally                         0.000040
         literal_unroll                         0.000022
         reconstruct_ssa                        0.004113
         LiteralPropagation                     0.000025
         nopython_type_inference                0.005242
         strip_phis                             0.001143
         inline_overloads                       0.000166
         nopython_rewrites                      0.002710
         nopython_supported_feature_validation  0.000073
         ir_legalization                        0.001064
         annotate_types                         0.000046
         native_lowering                        0.038367
         nopython_backend                       0.000026
         dump_parfor_diagnostics                0.000003

In [52]:
df_rvsdg["time"].sum() / df_baseline["time"].sum()

1.4088836554103545

In [53]:
import hvplot.pandas

df.hvplot.bar(stacked=True, width=600, height=1000, cmap="TolRainbow")

:Bars   [run,pass]   (time)

In [54]:
# Combine these plot so they are in the same scale
df_baseline[["pass", "time"]].set_index("pass").hvplot.barh(width=400, height=600) +\
df_rvsdg[["pass", "time"]].set_index("pass").hvplot.barh(width=400, height=600)

:Layout
   .Bars.I  :Bars   [pass]   (time)
   .Bars.II :Bars   [pass]   (time)